In [2]:
import numpy as np
from tqdm import tqdm

In [3]:
#define Pauli-Z, Pauli-X and identity matrices
pZ = np.array([[1,0],[0,-1]])
pX = np.array([[0,1],[1,0]])
eye = np.array([[1,0],[0,1]])

#define intereaction lists
Hlist = [(0,3),(3,6),(1,4),(4,7),(2,5),(5,8)]
Vlist = [(0,1,2),(1,2,0),(2,0,1),(3,4,5),(4,5,3),(5,3,4),(6,7,8),(7,8,6),(8,6,7)]
J = 1.0
g = 1.7
h = 0.7

In [4]:
#read nth bit
def ReadBit(i,n):
    return (i&(1<<n))>>n

#define many_body_operator
def many_body_operator(idx, oprts, size = 9):
    "Tensor product of `orts` acting on indexes `idx`. Fills rest with Id."
    matrices = [eye if k not in idx else oprts[idx.index(k)] for k in range(size)]
    prod = matrices[0]
    for k in range(1, size):
        prod = np.kron(prod, matrices[k]) 
    return prod

#define contraction of physical index of MPOs, we only need to calculate M0, M1 and M2, 
#(M0 = M3 = M6, M1 = M4 = M7, M2 = M5 = M8 due to translational invariance)
def PhysContractM0(lbit,rbit):
    i = eye[lbit,rbit]
    z = pZ[lbit,rbit]
    x = pX[lbit,rbit]
    M0 = np.array([[i,z,0,0,x,0,-h*x],
                    [0,0,i,0,0,0,0],
                    [0,0,0,i,0,0,0],
                    [0,0,0,0,0,0,-J*z],
                    [0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,i]])
    return M0+0j

def PhysContractM1(lbit,rbit):
    i = eye[lbit,rbit]
    z = pZ[lbit,rbit]
    x = pX[lbit,rbit]
    M1 = np.array([[i,z,0,0,0,0,-h*x],
                    [0,0,i,0,z,x,0],
                    [0,0,0,i,0,0,0],
                    [0,0,0,0,0,0,-J*z],
                    [0,0,0,0,0,z,0],
                    [0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,i]])
    return M1+0j

def PhysContractM2(lbit,rbit):
    i = eye[lbit,rbit]
    z = pZ[lbit,rbit]
    x = pX[lbit,rbit]
    M2 = np.array([[i,z,0,0,0,0,-h*x],
                    [0,0,i,0,0,0,0],
                    [0,0,0,i,0,0,0],
                    [0,0,0,0,0,0,-J*z],
                    [0,0,0,0,0,0,-g*x],
                    [0,0,0,0,0,0,-g*z],
                    [0,0,0,0,0,0,i]])
    return M2+0j

In [5]:
Hamil_contractmpo = np.zeros([2**9,2**9])+0j
Lvec = np.array([1,0,0,0,0,0,0])+0j
Rvec = np.transpose(np.array([0,0,0,0,0,0,1]))+0j

#Contract mpo to get the matrix elements of Hamiltonian
for i in tqdm(range(2**9)):
    for j in range(2**9):
        mel = Lvec @ PhysContractM0(ReadBit(i,0),ReadBit(j,0)) @ PhysContractM1(ReadBit(i,1),ReadBit(j,1)) @ \
        PhysContractM2(ReadBit(i,2),ReadBit(j,2)) @ PhysContractM0(ReadBit(i,3),ReadBit(j,3)) @ PhysContractM1(ReadBit(i,4),ReadBit(j,4))\
        @ PhysContractM2(ReadBit(i,5),ReadBit(j,5)) @ PhysContractM0(ReadBit(i,6),ReadBit(j,6)) @\
        PhysContractM1(ReadBit(i,7),ReadBit(j,7)) @ PhysContractM2(ReadBit(i,8),ReadBit(j,8)) @ Rvec
        Hamil_contractmpo[i,j] = mel

# Diagonalize the Hamiltonian
evalsmpo, evecsmpo = np.linalg.eigh(Hamil_contractmpo)

# Find the 20 lowest eigenvalues
lowest_evals = np.sort(evalsmpo)[:20]

print("The Lowest 20 Eigenvalues:{}".format(lowest_evals))

100%|████████████████████████████████████████████████████████████████████████████████| 512/512 [01:45<00:00,  4.83it/s]


The Lowest 20 Eigenvalues:[-16.98738797 -16.33765141 -14.20762975 -13.75785526 -13.51871353
 -13.51871353 -12.92917989 -12.47041377 -12.47041377 -12.35546297
 -12.35546297 -12.24793588 -11.50149999 -11.50149999 -11.22721333
 -11.22721333 -11.047253   -11.047253   -10.86124774 -10.55957422]


In [6]:
Hamil = np.zeros([2**9,2**9])+0j
for hh in Hlist:
    Hamil += -J*many_body_operator(hh, [pZ,pZ])
for v in Vlist:
    Hamil += -g*many_body_operator(v, [pZ,pX,pZ])
for i in range(9):
    Hamil += -h*many_body_operator([i], [pX])

In [7]:
# Diagonalize the Hamiltonian
evals, evecs = np.linalg.eigh(Hamil)

# Find the 20 lowest eigenvalues
lowest_evals = np.sort(evals)[:20]

print("The Lowest 20 Eigenvalues:{}".format(lowest_evals))

The Lowest 20 Eigenvalues:[-16.98738797 -16.33765141 -14.20762975 -13.75785526 -13.51871353
 -13.51871353 -12.92917989 -12.47041377 -12.47041377 -12.35546297
 -12.35546297 -12.24793588 -11.50149999 -11.50149999 -11.22721333
 -11.22721333 -11.047253   -11.047253   -10.86124774 -10.55957422]


In [8]:
#determine whether the contraction of mpos equals to the true Hamiltonian
print(np.array_equal(Hamil_contractmpo, Hamil))

True
